In [13]:
from dotenv import load_dotenv
import os
from google.colab import drive
drive.mount('/content/drive')
load_dotenv()

CREDENTIALS = {
'user' : os.getenv('ORACLE_USER'),
'password' : os.getenv('ORACLE_PASSWORD'),
'dsn' : os.getenv('ORACLE_DSN')
}

USER = CREDENTIALS['user']
PASS = CREDENTIALS["password"]
DSN = CREDENTIALS["dsn"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install oracledb
import pandas as pd
import oracledb
import locale
from datetime import datetime


# Tenta configurar locale para PT-BR (ajuda a ler datas)
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
except:
    print("⚠️ Locale PT-BR não configurado. Usaremos parsing manual de meses.")

⚠️ Locale PT-BR não configurado. Usaremos parsing manual de meses.


In [15]:
# Dicionário de Meses e Estados
meses_pt = {
    'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04', 'maio': '05', 'junho': '06',
    'julho': '07', 'agosto': '08', 'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12',
    'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06',
    'jul': '07', 'ago': '08', 'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
}

estados_uf = {
        'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Brasil': 'BR', 'Bahia': 'BA',
        'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
        'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
        'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
        'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
        'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
        'Sergipe': 'SE', 'Tocantins': 'TO'
}

In [16]:
arquivos_ibge = [
    {'arquivo': '/content/drive/MyDrive/FIAP/PMC.csv', 'indicador': 'PMC', 'skip': 3},
    {'arquivo': '/content/drive/MyDrive/FIAP/PMS.csv', 'indicador': 'PMS', 'skip': 3},
    {'arquivo': '/content/drive/MyDrive/FIAP/PIM.csv', 'indicador': 'PIM', 'skip': 3},
    {'arquivo': '/content/drive/MyDrive/FIAP/IPCA.csv', 'indicador': 'IPCA', 'skip': 3},
    {'arquivo': '/content/drive/MyDrive/FIAP/Desemprego.csv', 'indicador': 'DESEMPREGO', 'skip': 3}
]

dfs_finais = []

In [17]:
def tratar_data_ibge(texto_data):
    # Ex: "janeiro 2024" -> 01/01/2024
    # Ex: "nov-dez-jan 2024" -> 01/01/2024 (Pega o último mês do trimestre)
    try:
        partes = texto_data.split(' ')
        ano = partes[-1]
        mes_texto = partes[0].lower()

        # Se for trimestre móvel (ex: "nov-dez-jan"), pega o último
        if '-' in mes_texto:
            mes_texto = mes_texto.split('-')[-1]

        mes_num = meses_pt.get(mes_texto, '01')
        return pd.to_datetime(f"{ano}-{mes_num}-01")
    except:
        return None

In [18]:
for item in arquivos_ibge:
    print(f"📖 Processando {item['indicador']}...")
    try:
        # Lê o CSV ignorando as linhas de cabeçalho extras
        df_ibge = pd.read_csv(item['arquivo'], skiprows=item['skip'], encoding='utf-8', sep=',')

        # Identifica a coluna de Estado (geralmente a primeira, mas PIM chama diferente)
        col_uf = df_ibge.columns[0]

        # Melt (Transforma colunas de datas em linhas)
        df_melt = df_ibge.melt(id_vars=[col_uf], var_name='data_texto', value_name='vl_indicador')

        # Tratamento
        df_melt['nm_indicador'] = item['indicador']
        df_melt['dt_referencia'] = df_melt['data_texto'].apply(tratar_data_ibge)

        # Padroniza UF (Brasil -> BR)
        df_melt['sg_uf'] = df_melt[col_uf]
        df_melt['sg_uf'] = df_melt['sg_uf'].map(estados_uf)

        # Remove linhas com datas inválidas ou valores nulos/texto
        df_melt = df_melt.dropna(subset=['dt_referencia'])
        df_melt['vl_indicador'] = pd.to_numeric(df_melt['vl_indicador'], errors='coerce')
        df_melt = df_melt.dropna(subset=['vl_indicador'])

        dfs_finais.append(df_melt[['sg_uf', 'dt_referencia', 'nm_indicador', 'vl_indicador']])

    except Exception as e:
        print(f"❌ Erro ao ler {item['arquivo']}: {e}")

📖 Processando PMC...
📖 Processando PMS...
📖 Processando PIM...
📖 Processando IPCA...
📖 Processando DESEMPREGO...


In [19]:
# SELIC (Diária)
print("📖 Processando SELIC...")
df_selic = pd.read_csv('/content/drive/MyDrive/FIAP/SELIC.csv', sep=';', decimal=',')
df_selic.rename(columns={df_selic.columns[0]: 'dt_referencia', df_selic.columns[1]: 'vl_indicador'}, inplace=True)
df_selic.drop(df_selic.index[-1], inplace = True)
df_selic['dt_referencia'] = pd.to_datetime(df_selic['dt_referencia'], format='%d/%m/%Y', errors = 'coerce')
df_selic['vl_indicador'] = df_selic['vl_indicador'].str.replace(',', '.')
df_selic['vl_indicador'] = df_selic['vl_indicador'].astype('float')
df_selic_mensal = df_selic.set_index('dt_referencia').resample('MS')['vl_indicador'].mean().reset_index()
df_selic_mensal['nm_indicador'] = 'SELIC'
df_selic_mensal['sg_uf'] = 'BR'
dfs_finais.append(df_selic_mensal[['sg_uf', 'dt_referencia', 'nm_indicador', 'vl_indicador']])

📖 Processando SELIC...


In [20]:
# PTAX e IBC-Br (Arquivo Combinado)
print("📖 Processando PTAX e IBC-Br...")
df_ptax = pd.read_csv('/content/drive/MyDrive/FIAP/PTAX.csv', sep=';')
df_ibc = pd.read_csv('/content/drive/MyDrive/FIAP/IBC_Br.csv', sep=';')

# Prepara Dólar
df_ptax.columns = ['dt_referencia', 'vl_indicador']
df_ptax['nm_indicador'] = 'DOLAR'
df_ptax['sg_uf'] = 'BR'
df_ptax.drop(df_ptax.index[-1], inplace = True)
dfs_finais.append(df_ptax[['sg_uf', 'dt_referencia', 'nm_indicador', 'vl_indicador']])

# Prepara IBC-Br (PIB)
df_ibc.columns = ['dt_referencia', 'vl_indicador']
df_ibc['nm_indicador'] = 'PIB' # Ou 'IBCR', conforme sua view
df_ibc['sg_uf'] = 'BR'
df_ibc.drop(df_ibc.index[-1], inplace = True)
dfs_finais.append(df_ibc[['sg_uf', 'dt_referencia', 'nm_indicador', 'vl_indicador']])

# Ajusta datas do arquivo misto (MM/YYYY)
for df in dfs_finais[-2:]: # Pega os dois últimos adicionados
    df['dt_referencia'] = pd.to_datetime(df['dt_referencia'], format='%m/%Y')

📖 Processando PTAX e IBC-Br...


In [21]:
# INSERÇÃO DOS DADOS NO ORACLE
print("\n🔌 Conectando ao Oracle para carga...")
try:
    with oracledb.connect(user=USER, password=PASS, dsn=DSN) as connection:
        cursor = connection.cursor()
        print("🔌Conectou")

        # Limpa dados antigos (bom para os testes)
        cursor.execute("TRUNCATE TABLE T_BF_MACRO_ECONOMIA")
        print("🧹 Tabela limpa.")

        sql_insert = """
            INSERT INTO T_BF_MACRO_ECONOMIA (sg_uf, dt_referencia, nm_indicador, vl_indicador)
            VALUES (:1, :2, :3, :4)
        """

        total_inserido = 0
        for i, df in enumerate(dfs_finais):
            dados = df.values.tolist()
            cursor.executemany(sql_insert, dados)
            total_inserido += len(dados)
            print(f"   -> Lote {i+1}: {len(dados)} linhas inseridas.")

        connection.commit()
        print(f"✅ SUCESSO TOTAL! {total_inserido} indicadores carregados.")

except Exception as e:
    print(f"❌ Erro no Banco: {e}")


🔌 Conectando ao Oracle para carga...
🔌Conectou
🧹 Tabela limpa.
   -> Lote 1: 648 linhas inseridas.
   -> Lote 2: 648 linhas inseridas.
   -> Lote 3: 432 linhas inseridas.
   -> Lote 4: 24 linhas inseridas.
   -> Lote 5: 20 linhas inseridas.
   -> Lote 6: 36 linhas inseridas.
   -> Lote 7: 35 linhas inseridas.
   -> Lote 8: 34 linhas inseridas.
✅ SUCESSO TOTAL! 1877 indicadores carregados.
